
# 🧠 AI Lactate Training — **Complete Merged Notebook** (Lactate + Recovery)
**Colab-ready, one-click pipeline.**  
This notebook will:
- ✅ Load existing models if found (`models/*.joblib`) — else **auto-train**
- 🩸 Train **Lactate** model on wearable/effort data
- 🧬 Train **Recovery** model on **biomarkers + wearables**
- 💡 Produce SHAP explainability plots for both
- ☁️ Upload models to **GitHub** (`indarss/AI-Lactate-Advisor`) with `GITHUB_TOKEN`
- 🚀 Ping **Streamlit Cloud** to redeploy your app
- 🧪 If datasets are missing, it will **generate mock data** so the demo still runs


In [ ]:

# Install dependencies (quiet)
!pip install -q lightgbm shap PyGithub plotly joblib scikit-learn pandas numpy matplotlib


In [ ]:

import os, pathlib, json, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
import shap, joblib
from github import Github
import plotly.express as px

MODELS_DIR = "models"
DATA_DIR = "data"
pathlib.Path(MODELS_DIR).mkdir(exist_ok=True, parents=True)
pathlib.Path(DATA_DIR).mkdir(exist_ok=True, parents=True)

LACTATE_MODEL_PATH = os.path.join(MODELS_DIR, "lactate_lightgbm_model.joblib")
RECOVERY_MODEL_PATH = os.path.join(MODELS_DIR, "recovery_lightgbm_model.joblib")

LACTATE_CSV = os.path.join(DATA_DIR, "athlete_training_dataset_1000.csv")
BIOMARKER_CSV = os.path.join(DATA_DIR, "athlete_training_dataset_with_biomarkers.csv")


## 📥 Load Datasets (or auto-generate if missing)

In [ ]:

def synthesize_lactate_csv(path=LACTATE_CSV, n=1000, seed=42):
    np.random.seed(seed)
    df = pd.DataFrame({
        "time": np.arange(0, n*10, 10),
        "heart_rate": np.clip(np.linspace(110, 185, n) + np.random.normal(0, 3, n), 100, 190),
        "power": np.clip(np.linspace(100, 400, n) + np.random.normal(0, 10, n), 0, 450),
        "cadence": np.clip(np.linspace(75, 100, n) + np.random.normal(0, 2, n), 60, 120),
        "temperature": np.linspace(18, 25, n) + np.random.normal(0, 0.2, n),
        "altitude": np.linspace(100, 180, n) + np.random.normal(0, 1, n),
    })
    df["hr_slope_time"] = df["heart_rate"].diff() / df["time"].diff()
    df["hr_slope_power"] = df["heart_rate"].diff() / df["power"].diff()
    df["lactate"] = (
        1.3 +
        0.015 * (df["heart_rate"] - 120) +
        0.008 * (df["power"] - 150) / 10 +
        3 * np.maximum(df["hr_slope_time"], 0) +
        np.random.normal(0, 0.2, n)
    ).clip(1, 10)
    df.to_csv(path, index=False)
    return df

def synthesize_biomarker_csv(path=BIOMARKER_CSV, n=1000, seed=42):
    np.random.seed(seed)
    df = pd.DataFrame({
        "time": np.arange(0, n*10, 10),
        "heart_rate": np.clip(np.linspace(110, 185, n) + np.random.normal(0, 3, n), 100, 190),
        "power": np.clip(np.linspace(100, 400, n) + np.random.normal(0, 10, n), 0, 450),
        "cadence": np.clip(np.linspace(75, 100, n) + np.random.normal(0, 2, n), 60, 120),
        "temperature": np.linspace(18, 25, n) + np.random.normal(0, 0.2, n),
        "altitude": np.linspace(100, 180, n) + np.random.normal(0, 1, n),
    })
    df["hr_slope_time"] = df["heart_rate"].diff() / df["time"].diff()
    df["hr_slope_power"] = df["heart_rate"].diff() / df["power"].diff()

    # Biomarkers
    df["CK"] = np.random.normal(200, 50, n).clip(50, 600)
    df["Cortisol"] = np.random.normal(18, 4, n).clip(5, 35)
    df["T_C_ratio"] = np.random.normal(0.04, 0.01, n).clip(0.02, 0.08)
    df["hsCRP"] = np.random.normal(1.2, 0.5, n).clip(0.1, 5.0)
    df["Glucose"] = np.random.normal(90, 10, n).clip(70, 110)
    df["RBC"] = np.random.normal(4.8, 0.3, n).clip(4.0, 5.5)

    df["lactate"] = (
        1.3 +
        0.015 * (df["heart_rate"] - 120) +
        0.008 * (df["power"] - 150) / 10 +
        3 * np.maximum(df["hr_slope_time"], 0) +
        np.random.normal(0, 0.2, n)
    ).clip(1, 10)

    score = (
        100
        - 0.05 * (df["CK"] - 200)
        - 1.5 * (df["Cortisol"] - 18)
        - 1000 * (0.05 - df["T_C_ratio"])
        - 8 * (df["hsCRP"] - 1)
        - 0.2 * np.maximum(df["hr_slope_time"], 0) * 1000
        + 0.1 * (df["RBC"] - 4.8) * 100
    )
    df["recovery_score"] = np.clip(score, 0, 100)

    df.to_csv(path, index=False)
    return df

# Load or synthesize
if os.path.exists(LACTATE_CSV):
    df_lac = pd.read_csv(LACTATE_CSV)
    print("Loaded lactate dataset:", df_lac.shape)
else:
    print("Lactate dataset missing — synthesizing...")
    df_lac = synthesize_lactate_csv()
    print("Created:", df_lac.shape, "->", LACTATE_CSV)

if os.path.exists(BIOMARKER_CSV):
    df_rec = pd.read_csv(BIOMARKER_CSV)
    print("Loaded biomarker dataset:", df_rec.shape)
else:
    print("Biomarker dataset missing — synthesizing...")
    df_rec = synthesize_biomarker_csv()
    print("Created:", df_rec.shape, "->", BIOMARKER_CSV)

display(df_lac.head())
display(df_rec.head())


## 🩸 Lactate Model — Load or Train

In [ ]:

def train_lactate(df):
    X = df.drop(columns=["lactate"], errors="ignore")
    y = df["lactate"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = LGBMRegressor(n_estimators=200, learning_rate=0.05, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print("Lactate R²:", round(r2_score(y_test, y_pred), 3))
    print("Lactate MAE:", round(mean_absolute_error(y_test, y_pred), 3))
    return model, X_train, X_test, y_train, y_test

if os.path.exists(LACTATE_MODEL_PATH):
    try:
        lactate_model = joblib.load(LACTATE_MODEL_PATH)
        print("Loaded existing lactate model:", LACTATE_MODEL_PATH)
        X = df_lac.drop(columns=["lactate"], errors="ignore")
        y = df_lac["lactate"]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    except Exception as e:
        print("Failed to load existing model, retraining.", e)
        lactate_model, X_train, X_test, y_train, y_test = train_lactate(df_lac)
        joblib.dump(lactate_model, LACTATE_MODEL_PATH)
        print("Saved:", LACTATE_MODEL_PATH)
else:
    lactate_model, X_train, X_test, y_train, y_test = train_lactate(df_lac)
    joblib.dump(lactate_model, LACTATE_MODEL_PATH)
    print("Saved:", LACTATE_MODEL_PATH)


### 💡 Lactate SHAP & 3D Visualization

In [ ]:

explainer_lac = shap.Explainer(lactate_model, X_train.sample(min(1000, len(X_train)), random_state=42))
shap_values_lac = explainer_lac(X_test.sample(min(500, len(X_test)), random_state=42))
plt.figure(figsize=(8,6))
shap.summary_plot(shap_values_lac, X_test, plot_type="bar", show=False)
plt.title("Lactate Model — SHAP Global Feature Importance")
plt.tight_layout()
plt.show()

fig = px.scatter_3d(
    df_lac, x="heart_rate", y="power", z="lactate",
    color="lactate", color_continuous_scale="Turbo",
    size=("hr_slope_time" if "hr_slope_time" in df_lac.columns else None),
    title="3D Relationship: Heart Rate × Power × Lactate", opacity=0.85
)
fig.update_layout(scene=dict(xaxis_title='Heart Rate (bpm)', yaxis_title='Power (W)', zaxis_title='Lactate (mmol/L)'))
fig.show()


## 🧬 Recovery Model — Load or Train (Biomarkers + Wearables)

In [ ]:

def train_recovery(df):
    Xr = df.drop(columns=["recovery_score"], errors="ignore")
    yr = df["recovery_score"]
    Xr_train, Xr_test, yr_train, yr_test = train_test_split(Xr, yr, test_size=0.2, random_state=42)
    model = LGBMRegressor(n_estimators=300, learning_rate=0.05, random_state=42)
    model.fit(Xr_train, yr_train)
    yr_pred = model.predict(Xr_test)
    print("Recovery R²:", round(r2_score(yr_test, yr_pred), 3))
    print("Recovery MAE:", round(mean_absolute_error(yr_test, yr_pred), 3))
    return model, Xr_train, Xr_test, yr_train, yr_test

if os.path.exists(RECOVERY_MODEL_PATH):
    try:
        recovery_model = joblib.load(RECOVERY_MODEL_PATH)
        print("Loaded existing recovery model:", RECOVERY_MODEL_PATH)
        Xr = df_rec.drop(columns=["recovery_score"], errors="ignore")
        yr = df_rec["recovery_score"]
        Xr_train, Xr_test, yr_train, yr_test = train_test_split(Xr, yr, test_size=0.2, random_state=42)
    except Exception as e:
        print("Failed to load existing recovery model, retraining.", e)
        recovery_model, Xr_train, Xr_test, yr_train, yr_test = train_recovery(df_rec)
        joblib.dump(recovery_model, RECOVERY_MODEL_PATH)
        print("Saved:", RECOVERY_MODEL_PATH)
else:
    recovery_model, Xr_train, Xr_test, yr_train, yr_test = train_recovery(df_rec)
    joblib.dump(recovery_model, RECOVERY_MODEL_PATH)
    print("Saved:", RECOVERY_MODEL_PATH)


### 💡 Recovery SHAP

In [ ]:

explainer_rec = shap.Explainer(recovery_model, Xr_train.sample(min(1000, len(Xr_train)), random_state=42))
shap_values_rec = explainer_rec(Xr_test.sample(min(500, len(Xr_test)), random_state=42))
plt.figure(figsize=(8,6))
shap.summary_plot(shap_values_rec, Xr_test, plot_type="bar", show=False)
plt.title("Recovery Model — SHAP Global Feature Importance")
plt.tight_layout()
plt.show()


## 💾 Save Models

In [ ]:

joblib.dump(lactate_model, LACTATE_MODEL_PATH)
joblib.dump(recovery_model, RECOVERY_MODEL_PATH)
print("Saved:", LACTATE_MODEL_PATH, "and", RECOVERY_MODEL_PATH)


## ☁️ Upload Both Models to GitHub (indarss/AI-Lactate-Advisor)

In [ ]:

GITHUB_USERNAME = "indarss"
GITHUB_REPO = "AI-Lactate-Advisor"
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")  # set in Colab: Runtime → Secrets

if not GITHUB_TOKEN:
    print("⚠️ GITHUB_TOKEN not set. Skipping GitHub upload.")
else:
    g = Github(GITHUB_TOKEN)
    repo = g.get_user().get_repo(GITHUB_REPO)

    def upload_or_update(local_path, repo_path, message):
        with open(local_path, "rb") as f:
            content = f.read()
        try:
            contents = repo.get_contents(repo_path)
            repo.update_file(contents.path, message, content, contents.sha, branch="main")
            print(f"✅ Updated {repo_path}")
        except Exception as e:
            repo.create_file(repo_path, message, content, branch="main")
            print(f"✅ Uploaded {repo_path}")

    upload_or_update(LACTATE_MODEL_PATH, "models/lactate_lightgbm_model.joblib", "Update lactate model")
    upload_or_update(RECOVERY_MODEL_PATH, "models/recovery_lightgbm_model.joblib", "Update recovery model")


## 🚀 Trigger Streamlit Cloud Redeploy

In [ ]:

import requests
STREAMLIT_APP_URL = "https://share.streamlit.io/indarss/AI-Lactate-Advisor/main/app.py"
print("Pinging Streamlit Cloud to trigger redeploy…")
try:
    r = requests.get(STREAMLIT_APP_URL, timeout=20)
    print("Status:", r.status_code)
    if r.status_code == 200:
        print("✅ App reachable. Redeploy likely triggered automatically.")
    else:
        print("ℹ️ App responded but may still be rebuilding.")
except Exception as e:
    print("⚠️ Could not ping Streamlit automatically. You can redeploy from Streamlit Cloud UI.")



---
### ✅ You’re done!
- Models saved locally under `models/`
- If `GITHUB_TOKEN` set, models synced to **indarss/AI-Lactate-Advisor**
- Streamlit app pinged to redeploy

> Tip: Keep your datasets under `data/` in your repo so the app and this notebook stay aligned.
